In [19]:
import os
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder


# Parameters
characters = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&()')
image_size = (200, 200)
input_shape = (*image_size, 4)  # 4 for RGBA
n_classes = len(characters)
batch_size = 16
epochs = 10

# Load the image data
image_files = []
labels = []

for char in characters:
    for num in range(2, 5):
        filename = f'distorted_outlined_{char}_{num}.png'
        if os.path.exists(filename):
            img = load_img(filename, target_size=image_size, color_mode='rgba')
            img_array = img_to_array(img)
            image_files.append(img_array)
            labels.append(char)


# One-hot encode the labels
encoder = OneHotEncoder()
labels = np.array(labels).reshape(-1, 1)
labels = encoder.fit_transform(labels).toarray()


# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(image_files, labels, test_size=0.2, random_state=42)

# Build the model
model = keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(n_classes, activation='softmax')
])

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit the model
model.fit(np.array(X_train), np.array(y_train), batch_size=batch_size, epochs=epochs, verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)))

# Evaluate the model
score = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/10
7/7 [==============================] - 25s 3s/step - loss: 116.2516 - accuracy: 0.0278 - val_loss: 3.7842 - val_accuracy: 0.1111
Epoch 2/10
7/7 [==============================] - 20s 3s/step - loss: 3.4016 - accuracy: 0.2222 - val_loss: 3.4876 - val_accuracy: 0.2593
Epoch 3/10
7/7 [==============================] - 20s 3s/step - loss: 2.2245 - accuracy: 0.5370 - val_loss: 2.0674 - val_accuracy: 0.5185
Epoch 4/10
7/7 [==============================] - 20s 3s/step - loss: 0.8109 - accuracy: 0.7870 - val_loss: 0.8748 - val_accuracy: 0.7407
Epoch 5/10
7/7 [==============================] - 20s 3s/step - loss: 0.3873 - accuracy: 0.8611 - val_loss: 0.3632 - val_accuracy: 0.8519
Epoch 6/10
7/7 [==============================] - 20s 3s/step - loss: 0.2446 - accuracy: 0.9167 - val_loss: 0.3151 - val_accuracy: 0.8889
Epoch 7/10
7/7 [==============================] - 20s 3s/step - loss: 0.1667 - accuracy: 0.9537 - val_loss: 0.3338 - val_accuracy: 0.8889
Epoch 8/10
7/7 [================

In [6]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder

characters = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!@#$%^&()')
# Parameters
image_size = (200, 200)
input_shape = (*image_size, 4)  # 4 for RGBA
n_classes = len(characters)
batch_size = 8
epochs = 15

# Load the image data
image_files = []
labels = []
for char in characters:
    for num in range(2, 5):
        filename = f'distorted_outlined_{char}_{num}.png'
        if os.path.exists(filename):
            img = load_img(filename, target_size=image_size, color_mode='rgba')
            img_array = img_to_array(img)
            image_files.append(img_array)
            labels.append(char)
encoder = OneHotEncoder()
labels = np.array(labels).reshape(-1, 1)
labels = encoder.fit_transform(labels).toarray()
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(image_files, labels, test_size=0.2, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=50,
    zoom_range=0.5,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.23,
    horizontal_flip=True,
    fill_mode="nearest")

# Build the model with increased complexity and regularization
model = keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(n_classes, activation='softmax')
])

# Compile the model with a lower learning rate
optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15)

# Fit the model with data augmentation and early stopping
model.fit(datagen.flow(np.array(X_train), np.array(y_train), batch_size=batch_size),
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          callbacks=[early_stopping])

# Evaluate the model
score = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


C:\Users\lbowe\AppData\Roaming\Python\Python38\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/15
18/18 [==============================] - 55s 3s/step - loss: 36.1441 - accuracy: 0.0139 - val_loss: 3.8101 - val_accuracy: 0.0278
Epoch 2/15
18/18 [==============================] - 39s 2s/step - loss: 3.8048 - accuracy: 0.0278 - val_loss: 3.8150 - val_accuracy: 0.0000e+00
Epoch 3/15
18/18 [==============================] - 37s 2s/step - loss: 3.8130 - accuracy: 0.0139 - val_loss: 3.8296 - val_accuracy: 0.0000e+00
Epoch 4/15
18/18 [==============================] - 38s 2s/step - loss: 3.7954 - accuracy: 0.0347 - val_loss: 3.8463 - val_accuracy: 0.0000e+00
Epoch 5/15
18/18 [==============================] - 39s 2s/step - loss: 3.8192 - accuracy: 0.0556 - val_loss: 3.8922 - val_accuracy: 0.0000e+00
Epoch 6/15
18/18 [==============================] - 38s 2s/step - loss: 3.8176 - accuracy: 0.0139 - val_loss: 3.8800 - val_accuracy: 0.0000e+00
Epoch 7/15
14/18 [======================>.......] - ETA: 8s - loss: 3.7971 - accuracy: 0.0357 

KeyboardInterrupt: 